In [1]:
# [Cell 1] 기본 라이브러리 및 설정
import os
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 설정값
INPUT_FILE = "project_full_context.txt"
OUTPUT_JSON = "project_flows.json"

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 실행 환경: {device.upper()}")

c:\Pyg\Projects\semi\yuzyproject-aimodels\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 실행 환경: CUDA


In [2]:
# [Cell 2] Qwen 2.5-Coder-1.5B 모델 로딩

# 모델 ID 변경 (Qwen 2.5 Coder 1.5B)
MODEL_ID = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

# GPU 확인
print(f"🚀 실행 환경: {'CUDA (GPU)' if torch.cuda.is_available() else 'CPU'}")

try:
    print(f"🔄 모델 로딩 중... ({MODEL_ID})")
    
    # 1. 토크나이저 로딩
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    
    # 2. 모델 로딩
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype=torch.float16, # GPU 메모리 절약
        trust_remote_code=True
    )
    
    # Qwen 모델은 pad_token이 없을 수 있어 강제로 설정해줘야 에러가 안 난다.
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    print("✅ 모델 로딩 완료!")

except Exception as e:
    print(f"❌ 모델 로딩 실패: {e}")
    print("💡 힌트: CUDA 에러가 났다면 반드시 'Kernel Restart'를 먼저 해야 합니다.")

🚀 실행 환경: CUDA (GPU)
🔄 모델 로딩 중... (Qwen/Qwen2.5-Coder-1.5B-Instruct)


`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 338/338 [00:00<00:00, 527.78it/s, Materializing param=model.norm.weight]                              


✅ 모델 로딩 완료!


In [3]:
# [Cell 3] 프롬프트 매니저 (Qwen 1.5B 최종 완결판)

SYSTEM_PROMPT = """
You are a 'Senior Backend Architect'.
Your task is to generate a **Deep & Precise Call Graph** JSON.

### 🚨 CRITICAL RULES (DO NOT BE LAZY)
1. **NEVER Leave Children Empty**: You MUST trace down to the **Service/Repository** and **Database (Mongoose)** layers.
   - Bad: `"children": []`
   - Good: `"children": [{"function": "repository...", "children": [{"function": "User.find..."}]}]`
2. **Trace Middleware**: If a route has `isAuth`, add it as the FIRST child node.
3. **Analyze Logic**: 
   - `signup`: findByUserid -> bcrypt -> User.save.
   - `post`: isAuth -> controller -> repository -> Post.find/save.
4. **No Recursion**: A function (`login`) CANNOT call itself (`login`).

### ONE-SHOT EXAMPLE (Follow this depth strictly!)
Input Code: 
`router.post('/post', isAuth, createPost)`
`function createPost() { ... postRepository.create(...) }`
`function create() { ... new Post(...).save() }`

Output JSON:
{
  "category": "post",
  "endpoints": [
    {
      "method": "POST",
      "url": "/post",
      "function": "createPost",
      "children": [
        { "function": "isAuth", "file": "middleware/auth.mjs", "description": "Auth Check", "children": [] },
        { "function": "create", "file": "data/post.mjs", "description": "Repository Logic", "children": [
            { "function": "Post.save()", "file": "mongoose", "description": "DB Insert", "children": [] }
          ] 
        }
      ]
    }
  ]
}

### JSON OUTPUT FORMAT
Return ONLY valid JSON. Structure:
{
  "api": [
    { "category": "auth", "categoryName": "Auth Feature", "endpoints": [...] },
    { "category": "post", "categoryName": "Post Feature", "endpoints": [...] }
  ]
}
"""

print("✅ Qwen 1.5B 최종 프롬프트(Post 깊이 강화) 설정 완료")

✅ Qwen 1.5B 최종 프롬프트(Post 깊이 강화) 설정 완료


In [4]:
# [Cell 4] 분석 실행 (Qwen 1.5B - 고속 정밀 모드 / Greedy Search)

def extract_json(text):
    text = re.sub(r"^```(json)?", "", text.strip(), flags=re.MULTILINE)
    text = re.sub(r"```$", "", text.strip(), flags=re.MULTILINE)
    start = text.find('{')
    end = text.rfind('}')
    if start == -1 or end == -1: return "{}"
    return text[start:end+1]

def run_full_scan():
    if not os.path.exists(INPUT_FILE):
        print(f"❌ 파일 없음: {INPUT_FILE}")
        return

    print(f"📂 코드 분석 시작: '{INPUT_FILE}'")
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        full_code = f.read()
    
    code_context = full_code[:50000] 

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Analyze the source code to visualize the logic flow. Generate the Nested JSON Structure:\n\n{code_context}"}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    print("🧠 Qwen 1.5B가 '고속 정밀 모드(Greedy)'로 분석 중입니다... (빠름 & 결과 고정)")

    # 빔 서치-> 그리디 서치
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=8000,
        # [중요] 랜덤 끄기 (결과 고정)
        do_sample=False,
        num_beams=1,
        # 반복 방지
        repetition_penalty=1.1,
        
        early_stopping=True
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    json_str = extract_json(response)
    
    try:
        data = json.loads(json_str)
        with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"✅ 분석 완료! -> {OUTPUT_JSON}")
        return data
    except:
        print("❌ JSON 파싱 실패.")
        return None

# 실행
json_data = run_full_scan()

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📂 코드 분석 시작: 'project_full_context.txt'
🧠 Qwen 1.5B가 '고속 정밀 모드(Greedy)'로 분석 중입니다... (빠름 & 결과 고정)
✅ 분석 완료! -> project_flows.json


In [5]:
# [Cell 5] 텍스트 트리 뷰어 (JSON 구조 확인용)

def print_tree(node, prefix="", is_last=True):
    """재귀적으로 JSON 트리를 출력"""
    connector = "└── " if is_last else "├── "
    
    # 노드 이름 결정 (function 또는 method+url)
    if "method" in node:
        name = f"[{node['method']}] {node['url']} ({node.get('description', '')})"
    elif "function" in node:
        name = f"ƒ {node['function']} - {node.get('description', '')}"
    elif "category" in node:
        name = f"📂 Category: {node.get('categoryName', node['category'])}"
    else:
        name = "Unknown Node"

    print(prefix + connector + name)
    
    # 자식 노드 순회
    children = node.get("children", [])
    if "endpoints" in node:
        children = node["endpoints"]
        
    count = len(children)
    for i, child in enumerate(children):
        new_prefix = prefix + ("    " if is_last else "│   ")
        print_tree(child, new_prefix, i == count - 1)

def visualize_json_structure():
    if not os.path.exists(OUTPUT_JSON):
        print("❌ JSON 파일이 없습니다.")
        return

    with open(OUTPUT_JSON, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print("\n🌳 API Call Graph Analysis Result\n" + "="*40)
    if "api" in data:
        for cat in data["api"]:
            print_tree(cat)
    else:
        print("⚠️ 'api' 키를 찾을 수 없습니다. JSON 구조를 확인하세요.")

# 실행
visualize_json_structure()


🌳 API Call Graph Analysis Result
└── 📂 Category: Auth Feature
    ├── [POST] /auth/signup ()
    │   ├── ƒ body validation - Validation of Signup Request Body
    │   ├── ƒ findByUserid - Check for Existing UserID
    │   ├── ƒ bcrypt hash - Hash Password
    │   ├── ƒ createUser - Create New User
    │   ├── ƒ createJwtToken - Generate JWT Token
    │   └── ƒ res.send - Send JWT Token and User ID
    ├── [POST] /auth/login ()
    │   ├── ƒ body validation - Validation of Login Request Body
    │   ├── ƒ findByUserid - Check for Existing UserID
    │   ├── ƒ bcrypt compare - Compare Passwords
    │   ├── ƒ createJwtToken - Generate JWT Token
    │   └── ƒ res.send - Send JWT Token
    └── [POST] /auth/me ()
        ├── ƒ isAuth - Authentication Check
        ├── ƒ findById - Find User by ID
        └── ƒ res.send - Send User Information
└── 📂 Category: Post Feature
    ├── [GET] /post ()
    │   ├── ƒ isAuth - Authentication Check
    │   ├── ƒ getAllByUserid - Get Posts by User ID
  